##벡터 데이터베이스 Faiss를 이용한 임베딩 검색기(Semantic Search)
시맨틱 검색 (Semantic search) 은 기존의 키워드 매칭이 아닌 문장의 의미에 초점을 맞춘 정보 검색 시스
템을 말합니다. 여기서는 SBERT 와 FAISS 를 사용하여 간단한 검색 엔진을 구현해봅시다. Faiss 는 벡터
화 된 데이터를 인덱싱하고 데이터에 대한 효율적인 검색을 수행하기 위해 Facebook AI 에서 구축한 C ++
기반 라이브러리입니다. 우선 필요한 라이브러리를 설치합니다. CPU 환경에서 진행하신다면 faiss‐gpu
가 아니라 faiss‐cpu 를 설치하시기 바랍니다

In [1]:
!pip install faiss-gpu
!pip install -U sentence-transformers

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)


###1. 데이터 로드

In [2]:
import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer

여기서는 약 100 만개의 뉴스 기사 제목 데이터를 사용합니다. 데이터를 로드하여 리스트 형태로 변환하
겠습니다.

In [3]:
urllib.request.urlretrieve("https://media.githubusercontent.com/media/Siddharthsuresh92/Topic-Modeling-LDA/refs/heads/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
df = pd.read_csv("abcnews-date-text.csv")
data = df.headline_text.to_list()

상위 5 개의 샘플을 출력해봅시다.

In [4]:
data[:5]

['aba decides against community broadcasting licence',
 'act fire witnesses must be aware of defamation',
 'a g calls for infrastructure protection summit',
 'air nz staff in aust strike for pay rise',
 'air nz strike to affect australian travellers']

샘플 수를 확인해보면 약 110 만개가 존재합니다.

In [5]:
print('총 샘플 수:', len(data))

총 샘플 수: 1103663


###2. SBERT 임베딩
모든 샘플에 대해서 SBERT로 임베딩을 진행합니다.

In [6]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(data)
print('임베딩 된 벡터 수: ', len(encoded_data))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


임베딩 된 벡터 수:  1103663


###3. 인덱스 정의 및 데이터 추가
인덱스를 정의 하고 여기에 데이터 추가

In [7]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(data))))

faiss.write_index(index, 'abc_news')

###4. 검색 및 시간 측정
실제 검색을 진행해보고 시간을 측정해 봅시다. 여기서 주어진 쿼리에 대해서 유사도가 높은 상위 5개의 샘플을 추출 하겠습니다.

In [8]:
def search(query):
  t = time.time()
  query_vector = model.encode([query])
  k = 5
  top_k = index.search(query_vector, k)
  print('total time: {}'.format(time.time()-t))
  return [data[_id] for _id in top_k[1][0]]

In [11]:
query = str(input())
results = search(query)

print('result :')
for result in results:
  print('\t', result)

Underwater Forest Discovered
total time: 0.28690171241760254
result :
	 underwater loop
	 thriving underwater antarctic garden discovered
	 baton goes underwater in wa
	 underwater footage shows inside doomed costa
	 underwater uluru found off wa coast


’Underwater Forest Discovered’ 라는 임의의 문장을 입력해보겠습니다.